# 数据导入与清洗

In [28]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg' # 矢量图

In [80]:
# 将抓取到的多个 CSV 文件合并成一个 csv 表
%%writefile apeendCSV.py
def append_df(campus_name):
    """将多个 csv 文件整合成一个 csv 文件，读取并返回 Pandas.DataFrame
    
    Arg:
        campus_name: 高校名称英文缩写（大小写均可）
    Return:
        包含合并后的全部信息的 Pandas.DataFrame.
    """
    import glob

    df = pd.DataFrame()
    for path in glob.glob('./uncleaned_data/data_'+str.lower(campus_name)+'/*.csv'):
        df = df.append(pd.read_csv(path, index_col='post_time', parse_dates=['post_time'], infer_datetime_format=True))
    df.to_csv('./uncleaned_data/data_'+str.lower(campus_name)+'/data_' + campus_name +'.csv',header=True,index=True)
    

Writing apeendCSV.py


In [102]:
datasets_path = './csv'
# _df_shu = pd.read_csv('./')
# del _df_shu
# df_shu = pd.read_csv(datasets_path + '/' + 'data_shu_raw.csv', index_col='post_time', parse_dates=['post_time'], infer_datetime_format=True)
# df_ruc = pd.read_csv("./csv/data_ruc.csv")
#  df_fdu = pd.read_csv("./csv/data_fdu.csv")



## 1.1 SHU 数据处理

In [103]:
!head -1 ./uncleaned_data/data_shu/Data_SHU.csv

,ID,Group,Articles,Prestige,Institute,Reply_Time,Content,Post_ID,Post_Title,Post_Time,Post_Position,Post_Number,Reply_Year,Reply_Month


从 `./uncleaned_data/data_shu/` 文件下导入数据，`./uncleaned_data` 存储了未清洗的数据，主要分为一下几步：

1. 由原始 `Reply_Time` 为索引导入数据，并格式化时序索引
2. 舍弃不需要的列，重命名列名
3. 删除 `post_id` 和 `thread_owner` 缺失的数据
4. 对发帖内容通过正则表达式检索找到所回复的对象 `target`，若无匹配，则默认回复给 `theard_owner`
5. 通过帖子标题和帖子发帖时间映射 `thread_uid`
5. 保存文件到 `./csv/data_shu_raw.csv` 下完成数据预处理
    

In [104]:
# 数据清洗与筛选

df_shu = pd.read_csv('./uncleaned_data/data_shu/Data_SHU.csv', index_col='Reply_Time', parse_dates=['Reply_Time'], infer_datetime_format=True)

df_shu.drop('Unnamed: 0',axis=1, inplace=True)

df_shu.index.name='post_time'

df_shu.drop(columns=['Articles','Prestige','Post_Number','Reply_Year','Reply_Month'],inplace=True)

df_shu.rename(columns={'ID':'post_id','Content':'content','Institute':'institution','Group':'group','Post_ID':'thread_owner','Post_Title':'thread_title','Post_Position':'board_name_cn','Post_Time':'thread_time'},inplace=True)

df_shu.dropna(axis=0,how='any',subset=['post_id','thread_owner'],inplace=True)


In [105]:
# 匹配回复对象 target

import re
_pat1 =  re.compile('以下是引用(.*?)在(?!0000)[0-9]{4}')
_pat2 = re.compile('引用:\( (.*?) @')
_pat3 = re.compile('引用 (.*?) 发表于')

# 测试
# str1 = '以下是引用masheng在2006-12-6'
# str2 = '引用:( mazecome @ 2007-01-31 08:32 查看原帖 )强烈建'
# str3 = '引用 №寶貝★兲使 发表于 2009/1/23 11:52:53 的话：大家都是女生阿，忽'


def find_target(x):
    s = str(x['content'])
    if re.search(_pat1, s):
        return re.search(_pat1, x['content']).group(1)
    if re.search(_pat2, s):
        return re.search(_pat2, x['content']).group(1)
    if re.search(_pat3, s):
        return re.search(_pat3, x['content']).group(1)
    else:
        return x['thread_owner']

df_shu['target'] = df_shu.apply(find_target, axis=1)# 匹配


In [106]:
# 映射 thread_uid
df_shu['thread_uid'] = df_shu.thread_time + df_shu.thread_title

In [107]:
# 检查数据
df_shu.head(3)

,post_id,group,institution,content,thread_owner,thread_title,thread_time,board_name_cn,target,thread_uid
post_time,,,,,,,,,,
2011-05-17 00:08:11,梅杨杨,校友用户,机电工程与自动化学院,10修是宝宝攻修~~~点修太费钱·~况且还要弄好多东西！帮战装备又不是多好的装备！每个帮有一...,恋上狼的兔子,[公告]梦幻西游online,2006-04-26 15:33:00,游戏天地,恋上狼的兔子,2006-04-26 15:33:00[公告]梦幻西游online
2011-05-29 18:24:44,stephylss,校友用户,影视艺术技术学院,额,上大魂,先上传一个马政经的试卷试试,2006-05-11 15:29:00,上课这点事 选课,上大魂,2006-05-11 15:29:00先上传一个马政经的试卷试试
2011-05-20 22:46:18,M.Schumacher,校友用户,Mercedes AMG Petronas,NaN,FerrariLege,【分享】美丽的上大～上大＝动物园＋植物园～,2006-05-17 12:21:00,伟长湖畔 灌水,FerrariLege,2006-05-17 12:21:00【分享】美丽的上大～上大＝动物园＋植物园～


In [108]:
# 数据保存到./csv/data_shu_raw.csv ./hdf/data_shu_raw.csv
df_shu.to_csv('./csv/data_shu.csv', header=True, index=True)
df_shu.to_pickle('./pickle/data_shu.pkl')

## 1.2 RUC 数据处理

In [85]:
!head -1 ./uncleaned_data/data_ruc/data_ruc.csv

post_time,post_id,board_name_cn,post_title,reply_id,url,target


In [110]:
df_ruc = pd.read_csv('./uncleaned_data/data_ruc/data_ruc.csv', index_col='post_time', parse_dates=['post_time'], infer_datetime_format=True)

df_ruc.rename(columns={'url':'thread_uid','board_name_cn':'board_name_en'},inplace=True)

df_ruc.dropna(subset=['post_id'], inplace=True)



In [111]:
df_ruc.head(5)

,post_id,board_name_en,post_title,reply_id,thread_uid,target
post_time,,,,,,
2011-09-02 09:18:26,TrueLies,Apple,从三家资费对比看北京联通秋季校园竞争力,None,article/Apple/5422,NaN
2011-09-04 16:07:28,Hydk,Apple,Re: 从三家资费对比看北京联通秋季校园竞争力,None,article/Apple/5422,TrueLies
2011-09-09 14:10:49,lixmin,Apple,Re: 从三家资费对比看北京联通秋季校园竞争力,None,article/Apple/5422,TrueLies
2008-10-23 22:48:41,safari,Apple,Leopard校内上网小白教程,None,article/Apple/4703,NaN
2008-10-27 15:35:03,heater,Apple,Re: Leopard校内上网小白教程,safari,article/Apple/4703,safari


<mark>可以看到 RUC 数据已经完成了 `target` 的匹配 ( `reply_id` 不为空则回复给 `reply_id`，否则则回复给 `post_id` )</mark>

In [112]:
# 数据保存到./csv/data_ruc_raw.csv
df_ruc.to_csv('./csv/data_ruc.csv', header=True, index=True)
df_ruc.to_pickle('./pickle/data_ruc.pkl')


## 1.3 PKU 数据处理

In [4]:
# 保存原始数据
df_pku = pd.read_csv('./uncleaned_data/data_pku/data_pku.csv', index_col='post_time', parse_dates=['post_time'], infer_datetime_format=True)
df_pku.rename(columns={'thread_url':'thread_uid','reply_id':'target'},inplace=True)
df_pku.to_csv('./csv/data_pku.csv',header=True, index=True)
df_pku.to_pickle('./pickle/data_pku.pkl')


## 1.4 FDU 数据处理 

In [84]:
# 保存原始数据
df_fdu = pd.read_csv('./uncleaned_data/data_fdu/data_fdu.csv', index_col='post_time', parse_dates=['post_time'], infer_datetime_format=True)
df_fdu.rename(columns={'reply_id':'target'},inplace=True)
df_fdu.to_csv('./csv/data_fdu.csv',header=True, index=True)
df_fdu.to_pickle('./pickle/data_fdu.pkl')